# Review Data from TransparentCalifornia

In [ ]:
import sys
!{sys.executable} -m pip install pandas matplotlib pandas_profiling genderize gender_guesser

import requests
from io import StringIO
import pandas as pd
from pandas_profiling import ProfileReport
import matplotlib
%matplotlib inline
from genderize import Genderize
import gender_guesser.detector as gender

You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
# Fetch data from publicpay.ca.gov
df = None
for year in range(2011,2020):    
    year_df = pd.read_csv(f'state-of-california-{year}.csv.zip', compression='zip')
    year_df = year_df[year_df['Base Pay'].ne('Not Provided')]
    year_df['Base Pay'] = pd.to_numeric(year_df['Base Pay']).astype('float64')
    year_df['Job Title'] = year_df['Job Title'].str.upper()
    if year == 2013:
        print(year_df)
    print(f'Year {year} Rows {len(year_df)}')
    df = year_df if df is None else df.append(year_df)

In [ ]:
print(df.columns)

### Profiling Report

In [ ]:
# profile = ProfileReport(df, title='Pandas Profiling Report', explorative=True)
# profile

### Top 20 Job Titles

In [ ]:
df['Job Title'].value_counts()[0:20].plot(kind='barh')

## Agencies

In [ ]:
df['Agency'].value_counts()[0:20].plot(kind='barh')

# Compute Genders

In [ ]:
df['First Name'] = df['Employee Name'].str.split(n=2).str[0]

In [ ]:
d = gender.Detector()
df['Gender'] = df['First Name'].apply(lambda name: d.get_gender(name.title()))

# Rate limited, so let's not use this for now
# Genderize().get(df['First Name'])

In [ ]:
df[['First Name', 'Gender']][0:100]

### Gender Distribution

In [ ]:
df['Gender'].value_counts()[0:100].plot(kind='barh')

In [ ]:
mean_salary_df = df.groupby(['Job Title', 'Gender'])['Base Pay'].mean()

In [ ]:
mean_salary_df

### Remove Duplicates
Remove any employee name that appears in a year more than once.

@TODO Review whether this step is necessary or not

In [ ]:
dupes_df = df.groupby(['Year', 'Employee Name'])['Base Pay'].count()

In [ ]:
dupes_df[dupes_df > 2]

In [ ]:
df = df.sort_values(['Year', 'Employee Name'])

In [ ]:
df_no_dupes = df.drop_duplicates(subset=['Year', 'Employee Name'], keep=False)

In [ ]:
df_no_dupes

In [ ]:
dupes_df2 = df_no_dupes.groupby(['Year', 'Employee Name'])['Base Pay'].count()

In [ ]:
dupes_df2[dupes_df2 > 2]

In [ ]:
df = df_no_dupes

### Add a year number column (based on years in the database, by employee name)

In [ ]:
df['Year Number'] = df.groupby(['Employee Name'])['Year'].rank("dense", ascending=True)

In [ ]:
df

In [ ]:
df.to_csv('all_years_with_gender.csv', index=True)